In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
from sklearn.model_selection import cross_val_score 
from math import sqrt
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('ISTrend_data.csv')
data

,Company,Time,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Classification
0,AMZN,2019-12-01,47635.0,1983.0,1515.0,89.65,2.78,2.89,0.00,2073.0,1983.0,1532.5,1
1,TSCO,2018-12-01,654.6,15.3,109.8,7.63,1.03,0.19,0.15,15.3,15.3,6.8,1
2,AZO,2019-08-01,642.6,81.8,279.7,59.14,15.19,5.43,0.00,80.9,81.8,44.7,1
3,EQT,2018-12-01,1934.8,221.7,-3753.1,3.36,-16.65,0.06,0.00,250.6,221.7,106.1,0
4,SOHU,2013-12-01,333.1,-57.8,-88.2,8.54,-2.32,-2.31,0.00,-45.1,-57.8,-88.1,1
5,DRI,2019-05-01,430.3,81.3,117.4,3.75,0.97,0.47,0.48,81.7,81.3,56.2,1
6,PH,2017-06-01,668.5,166.3,176.6,6.26,1.41,0.74,0.06,202.1,166.3,87.5,1
7,VRTX,2011-12-01,1267.2,916.3,784.2,6.16,3.91,2.81,0.00,916.3,916.3,564.3,0
8,PFG,2014-12-01,1188.1,513.6,231.4,4.02,0.71,0.76,0.30,515.5,513.6,223.7,1
9,DISH,2012-12-01,107.2,-618.2,-879.2,-0.08,-1.99,-0.74,0.00,-613.2,-618.2,-320.7,1


In [3]:
classes = data['Classification'] 
data = data.drop(['Company', 'Time', 'Classification'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")
    
# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Classification', classes)
data

Total Revenues
Done
Operating Income
Done
Net Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done


,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Classification
0,0.754239,0.212925,0.488938,0.037673,0.344334,0.165383,0.480023,0.214329,0.212925,0.249985,1
1,0.553576,0.196357,0.481506,0.032382,0.344310,0.165323,0.488709,0.197003,0.196357,0.230938,1
2,0.553525,0.196917,0.482404,0.035705,0.344506,0.165439,0.480023,0.197556,0.196917,0.231412,1
3,0.559044,0.198095,0.461074,0.032107,0.344065,0.165321,0.480023,0.198985,0.198095,0.232178,0
4,0.552203,0.195741,0.480458,0.032441,0.344264,0.165268,0.480023,0.196495,0.195741,0.229754,1
5,0.552618,0.196912,0.481546,0.032132,0.344309,0.165330,0.507817,0.197562,0.196912,0.231555,1
6,0.553636,0.197628,0.481859,0.032294,0.344315,0.165336,0.483497,0.198576,0.197628,0.231946,1
7,0.556193,0.203943,0.485073,0.032287,0.344350,0.165381,0.480023,0.204590,0.203943,0.237898,0
8,0.555855,0.200552,0.482149,0.032149,0.344305,0.165336,0.497394,0.201215,0.200552,0.233646,1
9,0.551238,0.191023,0.476275,0.031885,0.344268,0.165303,0.480023,0.191711,0.191023,0.226850,1


In [5]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:10] # Parameters
y = NpMatrix[:,10] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.75423917 0.21292468 0.48893807 ... 0.21432902 0.21292468 0.24998533]
 [0.55357628 0.19635667 0.48150562 ... 0.19700335 0.19635667 0.23093844]
 [0.55352502 0.1969166  0.48240426 ... 0.1975557  0.1969166  0.23141158]
 ...
 [0.5658876  0.19423231 0.47595825 ... 0.19441591 0.19423231 0.23161008]
 [0.53574144 0.17061424 0.4720971  ... 0.17126114 0.17061424 0.2105595 ]
 [0.55532448 0.19673389 0.48112056 ... 0.19751528 0.19673389 0.23125428]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 10
Total Number of Training instances: 4500


[1. 1. 1. ... 1. 0. 1.]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 1.0
Total number of y values 4500


In [6]:
params_RF = {'n_estimators':range(1,30),
            'max_depth' : range(1,15),
            }

cv_str = StratifiedKFold(n_splits=5)

RF = RandomForestClassifier()
gs_modelRF = GridSearchCV(RF, param_grid=params_RF, cv= cv_str)
grid_results = gs_modelRF.fit(X, y)

# Best parameters, best test score using these parameters, optimal KNN Configuration
print("Best Parameters", gs_modelRF.best_params_)
print("Best score on Test Data", gs_modelRF.best_score_)
print("Optimal Configuration", gs_modelRF.best_estimator_)

Best Parameters {'max_depth': 6, 'n_estimators': 28}
Best score on Test Data 0.7046666666666667
Optimal Configuration RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=28,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [7]:
# Summarize the results in a readable format
print("Best: {0}, using {1}".format(grid_results.cv_results_['mean_test_score'], grid_results.best_params_))
results_df = pd.DataFrame(grid_results.cv_results_)
results_df

Best: [0.67355556 0.67511111 0.67511111 0.67511111 0.67577778 0.67133333
 0.67511111 0.67644444 0.67511111 0.67511111 0.67511111 0.67511111
 0.67511111 0.67511111 0.67511111 0.67511111 0.67511111 0.67511111
 0.67511111 0.67511111 0.67511111 0.67511111 0.67511111 0.67511111
 0.67511111 0.67511111 0.67511111 0.67511111 0.67511111 0.68044444
 0.68644444 0.69177778 0.68466667 0.68933333 0.69222222 0.69577778
 0.69466667 0.69688889 0.69844444 0.69555556 0.68933333 0.69777778
 0.69088889 0.696      0.69466667 0.69711111 0.69488889 0.69044444
 0.696      0.69666667 0.69288889 0.696      0.69266667 0.69822222
 0.696      0.69466667 0.69622222 0.69955556 0.68333333 0.69355556
 0.694      0.69266667 0.69911111 0.69911111 0.69822222 0.69822222
 0.69488889 0.69311111 0.70133333 0.69711111 0.69511111 0.7
 0.69755556 0.69844444 0.69955556 0.70155556 0.69288889 0.70044444
 0.69933333 0.69888889 0.69666667 0.70044444 0.69955556 0.69866667
 0.69488889 0.69977778 0.70222222 0.67977778 0.692      0.69288

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002210,0.000391,0.000973,1.984204e-05,1,1,"{'max_depth': 1, 'n_estimators': 1}",0.674806,0.674806,0.667778,0.675195,0.675195,0.673556,0.002894,370
1,0.004176,0.000979,0.001009,9.067205e-04,1,2,"{'max_depth': 1, 'n_estimators': 2}",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,340
2,0.005001,0.000641,0.000591,4.825647e-04,1,3,"{'max_depth': 1, 'n_estimators': 3}",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,340
3,0.005786,0.000398,0.000997,5.722046e-07,1,4,"{'max_depth': 1, 'n_estimators': 4}",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,340
4,0.006783,0.000398,0.000997,7.599534e-07,1,5,"{'max_depth': 1, 'n_estimators': 5}",0.674806,0.674806,0.675556,0.678532,0.675195,0.675778,0.001404,338
5,0.008177,0.000401,0.000989,1.555644e-05,1,6,"{'max_depth': 1, 'n_estimators': 6}",0.674806,0.674806,0.675556,0.675195,0.656285,0.671333,0.007524,372
6,0.009582,0.000495,0.000991,1.491472e-05,1,7,"{'max_depth': 1, 'n_estimators': 7}",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,340
7,0.010771,0.000399,0.001396,4.883078e-04,1,8,"{'max_depth': 1, 'n_estimators': 8}",0.674806,0.674806,0.675556,0.681869,0.675195,0.676444,0.002725,336
8,0.012757,0.000396,0.001005,1.420543e-05,1,9,"{'max_depth': 1, 'n_estimators': 9}",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,340
9,0.014362,0.000789,0.001596,7.977009e-04,1,10,"{'max_depth': 1, 'n_estimators': 10}",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,340


In [8]:
results_df = results_df.sort_values(['rank_test_score'])
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
172,0.106549,2.225107e-03,0.004189,9.773791e-04,6,28,"{'max_depth': 6, 'n_estimators': 28}",0.711432,0.701443,0.711111,0.698554,0.700779,0.704667,0.005480,1
164,0.077180,1.022221e-03,0.002992,2.431402e-07,6,20,"{'max_depth': 6, 'n_estimators': 20}",0.711432,0.705882,0.710000,0.694105,0.696329,0.703556,0.007077,2
105,0.055843,1.888840e-03,0.002395,4.876265e-04,4,19,"{'max_depth': 4, 'n_estimators': 19}",0.709212,0.691454,0.708889,0.711902,0.695217,0.703333,0.008317,3
171,0.102932,2.302509e-03,0.003775,4.115292e-04,6,27,"{'max_depth': 6, 'n_estimators': 27}",0.713651,0.701443,0.707778,0.699666,0.692992,0.703111,0.007070,4
136,0.069415,4.727197e-04,0.002791,3.995916e-04,5,21,"{'max_depth': 5, 'n_estimators': 21}",0.702553,0.697003,0.712222,0.700779,0.703003,0.703111,0.005022,4
86,0.066817,6.316361e-04,0.003403,4.799862e-04,3,29,"{'max_depth': 3, 'n_estimators': 29}",0.705882,0.694784,0.706667,0.708565,0.695217,0.702222,0.005963,6
141,0.085798,1.224224e-03,0.003590,4.896318e-04,5,26,"{'max_depth': 5, 'n_estimators': 26}",0.703663,0.702553,0.711111,0.703003,0.690768,0.702222,0.006525,6
161,0.064426,1.013140e-03,0.002393,4.863888e-04,6,17,"{'max_depth': 6, 'n_estimators': 17}",0.705882,0.698113,0.708889,0.695217,0.700779,0.701778,0.004997,8
162,0.067819,6.313550e-04,0.002793,3.999748e-04,6,18,"{'max_depth': 6, 'n_estimators': 18}",0.700333,0.704772,0.706667,0.708565,0.688543,0.701778,0.007155,8
197,0.102326,1.196766e-03,0.002992,3.162980e-07,7,24,"{'max_depth': 7, 'n_estimators': 24}",0.703663,0.695893,0.711111,0.704116,0.694105,0.701778,0.006158,8
